In [3]:
delta_path = "s3a://real-estate/lake/bronze/property"
key = os.environ["MINIO_ROOT_USER"]
secret = os.environ["MINIO_SECRET_KEY"]
#endpoint = os.environ["MINIO_SECRET_ENDPOINT"]
endpoint = "http://127.0.0.1:9000" #"http://127.0.0.1:9000"

In [1]:
import os
import sys
import socket

import sys,uuid,datetime
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

#sc.stop()

spark = SparkSession.builder \
.master("k8s://https://kubernetes.docker.internal:6443") \
.appName("reading") \
.config("spark.hadoop.fs.s3a.access.key", key) \
.config("spark.hadoop.fs.s3a.secret.key", secret) \
.config("spark.hadoop.fs.s3a.endpoint", endpoint) \
.config("spark.hadoop.fs.s3a.path.style.access", "true") \
.config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
.config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0,org.apache.hadoop:hadoop-aws:3.2.0,com.amazonaws:aws-java-sdk-bundle:1.11.375") \
.config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
.config('spark.submit.deployMode', 'client') \
.config("spark.kubernetes.container.image", "spark:spark-docker") \
.config("spark.kubernetes.pyspark.pythonVersion", "3") \
.config("spark.kubernetes.authenticate.driver.serviceAccountName", "default") \
.config("spark.executor.instances", "2") \
.config("spark.executor.memory", "4g") \
.config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
.config("spark.kubernetes.executor.request.cores","0.5") \
.config("spark.kubernetes.executor.limit.cores","4") \
.config("jupyterService.jupyterPort", "30888") \
.config("serviceAccount", "spark") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.getOrCreate()

#.config("spark.driver.host", "10.1.2.104") \
#.config("spark.driver.port", "4040") \

sc = spark.sparkContext
#sc._conf.getAll()

192.168.46.93:9000


In [4]:
spark.sql("DESCRIBE FORMATTED delta.`{}`".format(delta_path)).show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|propertyDetails_a...|   bigint|       |
|propertyDetails_a...|   string|       |
|propertyDetails_a...|   string|       |
|propertyDetails_c...|   bigint|       |
|propertyDetails_c...|   string|       |
|propertyDetails_c...|   bigint|       |
|propertyDetails_c...|   bigint|       |
|propertyDetails_d...|   string|       |
|propertyDetails_g...|   bigint|       |
|propertyDetails_h...|  boolean|       |
|propertyDetails_h...|  boolean|       |
|  propertyDetails_id|   bigint|       |
|propertyDetails_i...|   string|       |
|propertyDetails_i...|  boolean|       |
|propertyDetails_i...|  boolean|       |
|propertyDetails_i...|   string|       |
|propertyDetails_i...|  boolean|       |
|propertyDetails_i...|  boolean|       |
|propertyDetails_i...|  boolean|       |
|propertyDetails_l...|   string|       |
+--------------------+---------+-------+
only showing top

In [5]:
spark.sql("DESCRIBE HISTORY delta.`{}`".format(delta_path)).show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|     22|2020-11-22 16:39:26|  null|    null|    MERGE|[predicate -> (tr...|null|    null|     null|         21|          null|        false|[numTargetRowsCop...|        null|
|     21|2020-11-22 16:38:06|  null|    null|    MERGE|[predicate -> (tr...|null|    null|     null|         20|          null|        false|[numTargetRowsCop...|        null|
|     20|2020-11-20 17:02:41|  null|    null|    MERGE|[predicate -> (tr...|null|    null|     null|         19|        

In [ ]:
spark.sql("SELECT COUNT(*) FROM delta.`{}`".format(delta_path)).show()

In [4]:
import pandas as pd
import json, ijson #made for big files which cannot be handled in memory

#visualisations
import folium
from folium import plugins

In [5]:
spark.sql("SELECT COUNT(*) FROM delta.`{}`".format(delta_path)).show()

+--------+
|count(1)|
+--------+
|     617|
+--------+



# Run Google Direction calculation
First we prepare Google Way Calculation. Attention, only 2500 request are free per day!

In [ ]:
from googleApiWayCalculation import calculateGoogleDirections

##
## read immoscout files to get property location to lookup in GoogleMaps
## --> Filter here must be changed at some point when Google API request are cheaper or less
##
def loadImmoProperties(file, path, filename):

    #city = file.split('_')[1]

    with open(path + file, 'r') as f:
        objects = ijson.items(f, 'item.propertyDetails')
        props = list(objects)

    p = []
    for prop in props:
        #flatten all dics/arrays inside 
        pp = flatten_json(prop)

        #TODO: change filter only in one place (jupyter notebook)
        if pp['propertyType'] in ['flat', 'penthouse', 'attic-flat', 'duplex-maisonette', 'detached-secondary-suite', 'loft', 'single-room']:
            p.append(flatten_json(prop))

    return p

## Here we run Google Way Calculation
define path, city (bigger city close by you want to go) and place to work. Script will calculate distance and how many times to change train.

In [ ]:
import datetime
path = '/Users/sspaeti/Simon/Sync/Business/13_sspaeti/Real-estate/_DATASETS/immo24/immo24_BS/'


workDestination = 'swissqual, Allmendweg 8, 4528 Zuchwil'

file = 'immo24ads_solothurn_buy_testing.json'
#file = 'immo24ads_bern_buy_2018-06-03_16-34-05.json'
#file = 'immo24ads_solothurn_buy_2018-05-31_19-31-16.json'

cityDestination = 'bern'
cityImmo = file.split('_')[1]
fileOutputName = 'googleDirections_' + cityImmo + '_tester.json'
"""fileOutputName = 'googleDirections_' + cityImmo + '_' + workDestination + '_{timestamp}.json'.format(
    timestamp=datetime.datetime.now().strftime(
        "%Y-%m-%d_%H-%M-%S")  # isoformat('YYYY-MM-DD HH:MM:SS)
)"""

#filters = {'propertyType': ['flat', 'penthouse', 'attic-flat', 'duplex-maisonette', 'detached-secondary-suite', 'loft', 'single-room']}



#get immo data
properties = loadImmoProperties(file, path, file)
calculateGoogleDirections(properties, cityDestination, workDestination, fileOutputName)

# Loadig data from Delta to Koalas and Pandas

In [8]:
import databricks.koalas as ks
delta_path = "s3a://real-estate/lake/bronze/property"
delta_table_name='property'
database = 'immo'

#koalas
dfprop = ks.read_delta(delta_path)

#pandas
dfprop_spark = spark.sql("SELECT * FROM delta.`{}`".format(delta_path))
#dfprop = dfprop_spark.toPandas()

#dfprop_spark.show()

dfprop_spark.printSchema()

dfprop.info()
#print(dfprop.index)
#dfprop.head(2)

df = dfprop

root
 |-- propertyDetails_accountId: long (nullable = true)
 |-- propertyDetails_availableFrom: string (nullable = true)
 |-- propertyDetails_availableFromFormatted: string (nullable = true)
 |-- propertyDetails_cityId: long (nullable = true)
 |-- propertyDetails_cityName: string (nullable = true)
 |-- propertyDetails_contactFormTypeId: long (nullable = true)
 |-- propertyDetails_countryId: long (nullable = true)
 |-- propertyDetails_description: string (nullable = true)
 |-- propertyDetails_geoAccuracy: long (nullable = true)
 |-- propertyDetails_hasNewBuildingProject: boolean (nullable = true)
 |-- propertyDetails_hasVirtualTour: boolean (nullable = true)
 |-- propertyDetails_id: long (nullable = true)
 |-- propertyDetails_isBuyRent: string (nullable = true)
 |-- propertyDetails_isHighlighted: boolean (nullable = true)
 |-- propertyDetails_isNew: boolean (nullable = true)
 |-- propertyDetails_isNewEndDate: string (nullable = true)
 |-- propertyDetails_isOnline: boolean (nullable = tr

In [ ]:
dfprop

# Loading different data sources
scraped Immoscout properties
google direction files

In [ ]:
#path = '/Users/sspaeti/Simon/Sync/Business/13_sspaeti/Real-estate/_DATASETS/immo24/immo24_BS/'
#fileImmoSo = 'immo24ads_solothurn_buy_2018-05-31_19-31-16.json'
#fileImmoBe = 'immo24ads_bern_buy_2018-06-03_16-34-05.json'
#fileImmoscout = 'immo24ads_solothurn_buy_testing.json'

def loadImmoScout(fileName, path):
    with open(path + fileName, 'r') as f:
        objects = ijson.items(f, 'item.propertyDetails')
        props = list(objects)    

    p = []
    for prop in props:
        #flatten all dics/arrays inside
        p.append(flatten_json(prop))
    
    #load into pandas dataFrame
    df = pd.DataFrame(p).set_index('id')
    return df

#dfprop = pd.concat([loadImmoScout(fileImmoBe, path), loadImmoScout(fileImmoSo, path)], sort=True)

### Loading Google Directions calculations

In [ ]:
fileGoogleSo = 'googleDirections_solothurn_swissqual, Allmendweg 8, 4528 Zuchwil_2018-06-03_23-36-21.json'
#fileGoogleDirections = 'googleDirections_solothurn_tester.json'
fileGoogleBe = 'googleDirections_bern_swissqual, Allmendweg 8, 4528 Zuchwil_2018-06-03_16-37-43.json'

def loadGoogleWayCalc(fileName, path):
    d = json.load( open( path + fileName, 'rb' ) )
    df = pd.DataFrame.from_dict(d)
    df = df.transpose() #switch cols and rows

    #convert integers
    df = df.apply(pd.to_numeric, errors='ignore')
    df.index.name='id'
    df = df.set_index('id')
    return df

#UNION ALL
dfWayCalc = pd.concat([loadGoogleWayCalc(fileGoogleBe, path), loadGoogleWayCalc(fileGoogleSo, path)])

#dfWayCalc.index.name = 'id'
dfWayCalc.info()
#dfWayCalc.index.name
#dfWayCalc.set_index('id', inplace=True)
#print('----1')
#print(dfWayCalc.index)
#print('----2')
#dfWayCalc.head(2)
#print(dfWayCalc.head(2))
#print('----3')
##access specific property:
#print(dfWayCalc.loc['4998203'])


### Merge datasets

In [ ]:
"""
print('****')
print(dfWayCalc.loc[4986917])

print('-------')

dfprop.info()
print('****')
dfprop.head(5)
print(dfprop.loc[4986917])

print('-------')
list(dfWayCalc)
list(dfprop)
"""

In [ ]:
df = dfprop.join(dfWayCalc, how='left')
df.info()
df.head(5)
#df.loc[4986917]

## Filtering properties

In [9]:
#reset filters
#df = dfprop.join(dfWayCalc, how='left')

##google way calcualation filters
##no houses
#df = df[df['propertyType'].isin(['flat','penthouse','attic-flat','duplex-maisonette','detached-secondary-suite','loft','single-room'])]
#no more changing trains than two times either to the city or work
#df = df[(df['transitsNoWalkingWork']<=2) & (df['transitsNoWalkingCity']<=2)]

#df = df[(df['routeDurationSecWork']<=50*60) & (df['routeDurationSecCity']<=60*60)]

df = df[df['propertyDetails_normalizedPrice']<=600000]
df.info()
#df.head(5)
#df

<class 'databricks.koalas.frame.DataFrame'>
Index: 250 entries, 1 to 614
Data columns (total 234 columns):
 #   Column                                                                  Non-Null Count  Dtype  
---  ------                                                                  --------------  -----  
 0   propertyDetails_accountId                                               250 non-null    int64  
 1   propertyDetails_availableFrom                                           78 non-null     object 
 2   propertyDetails_availableFromFormatted                                  250 non-null    object 
 3   propertyDetails_cityId                                                  250 non-null    int64  
 4   propertyDetails_cityName                                                250 non-null    object 
 5   propertyDetails_contactFormTypeId                                       250 non-null    int64  
 6   propertyDetails_countryId                                               250 non-

# Reporting
Top communes of this dataset

In [10]:
#AUSWERTUNG df
#df.loc[3975183] #direct id
#df.loc[3975183]['longitude']
print(df['propertyDetails_cityName'].value_counts().head(15))
#df.info()

Grenchen              31
Moutier               15
Bern                  11
Welschenrohr           9
Gerlafingen            8
Lengnau BE             8
Derendingen            7
Wiedlisbach            7
Etziken                7
Subingen               7
Recherswil             6
Solothurn              6
Luterbach              5
Wangen an der Aare     5
Alchenstorf            5
Name: propertyDetails_cityName, dtype: int64


In [32]:
ks.sql("SELECT * from {dfprop} WHERE propertyDetails_id = '6051892'")

,propertyDetails_accountId,propertyDetails_availableFrom,propertyDetails_availableFromFormatted,propertyDetails_cityId,propertyDetails_cityName,propertyDetails_contactFormTypeId,propertyDetails_countryId,propertyDetails_description,propertyDetails_geoAccuracy,propertyDetails_hasNewBuildingProject,propertyDetails_hasVirtualTour,propertyDetails_id,propertyDetails_isBuyRent,propertyDetails_isHighlighted,propertyDetails_isNew,propertyDetails_isNewEndDate,propertyDetails_isOnline,propertyDetails_isQualityListing,propertyDetails_isTopListing,propertyDetails_lastModified,propertyDetails_lastPublished,propertyDetails_latitude,propertyDetails_listingLayout,propertyDetails_longitude,propertyDetails_msRegionId,propertyDetails_municipalityBfsId,propertyDetails_municipalityId,propertyDetails_municipalityNameFormatted,propertyDetails_normalizedPrice,propertyDetails_normalizedPriceFormatted,propertyDetails_numberOfRooms,propertyDetails_numberOfRoomsFormatted,propertyDetails_offerTypeId,propertyDetails_packageType,propertyDetails_price,propertyDetails_priceFormatted,propertyDetails_priceUnitId,propertyDetails_priceUnitLabel,propertyDetails_propertyCategoryId,propertyDetails_propertyDetailUrl,propertyDetails_propertyId,propertyDetails_propertyType,propertyDetails_propertyTypeId,propertyDetails_referenceNo,propertyDetails_regionId,propertyDetails_searchCity,propertyDetails_searchDate,propertyDetails_searchDateTime,propertyDetails_searchRadius,propertyDetails_sellingPrice,propertyDetails_sellingPriceFormatted,propertyDetails_shortDescription,propertyDetails_showContactForm,propertyDetails_showContactFormAddress,propertyDetails_state,propertyDetails_stateShort,propertyDetails_street,propertyDetails_surfaceLiving,propertyDetails_surfaceLivingFormatted,propertyDetails_surfaceUsable,propertyDetails_surfaceUsableFormatted,propertyDetails_title,propertyDetails_url,propertyDetails_urlTitle,propertyDetails_videoViewingEnabled,propertyDetails_visitName,propertyDetails_visitPhone,propertyDetails_visitRemark,propertyDetails_zip,propertyDetails_zipId,viewData_agencyPhoneStatsUrl,viewData_allowOnlineApplication,viewData_areSimilarPropertiesFromSameAdvertiser,viewData_debtEnforcementLandingPageUrl,viewData_detailPrintStatsUrl,viewData_detailViewStatsUrl,viewData_financingLinkUrl,viewData_movingUrl,viewData_showAccountProperties,viewData_showSimilarProperties,viewData_visitPhoneStatsUrl,propertyDetails_agency_companyCity,propertyDetails_agency_companyName1,propertyDetails_agency_companyName2,propertyDetails_agency_companyPhoneBusiness,propertyDetails_agency_companyPhoneMobile,propertyDetails_agency_companyStreet,propertyDetails_agency_companyZip,propertyDetails_agency_firstName,propertyDetails_agency_gender,propertyDetails_agency_isAccountMigrated,propertyDetails_agency_isGuest,propertyDetails_agency_lastName,propertyDetails_agency_logoUrl,propertyDetails_agency_logoUrlDetailPage,propertyDetails_agency_nameFormatted,propertyDetails_agency_reference,propertyDetails_agency_showLogoOnSerp,propertyDetails_agency_userType,propertyDetails_agency_webUrl,propertyDetails_attributes_newBuilding,propertyDetails_attributes_oldBuilding,propertyDetails_attributes_underBuildingLaws,propertyDetails_attributes_yearBuilt,propertyDetails_attributes_yearRenovated,propertyDetails_attributesInside_animalAllowed,propertyDetails_attributesInside_cellar,propertyDetails_attributesInside_firePlace,propertyDetails_attributesInside_nrBathRooms,propertyDetails_attributesInside_propView,propertyDetails_attributesInside_reduit,propertyDetails_attributesInside_wheelChairAccessible,propertyDetails_attributesOutside_playGround,propertyDetails_attributesOutside_propBalcony,propertyDetails_attributesOutside_propChildFriendly,propertyDetails_attributesOutside_propElevator,propertyDetails_attributesOutside_propGarage,propertyDetails_attributesOutside_propParking,propertyDetails_attributesSize_floor,propertyDetails_attributesSize_floorFormatted,propertyDetails_attributesSize_numberOfFloors,propertyD

## Maps 
### All properties of this filtered dataset

In [11]:
file = "201115"
props_map = []
props_map = folium.Map(location=[47.208835, 7.532291],
                       zoom_start=11)
#marker_cluster = folium.plugins.MarkerCluster().add_to(props_map)
for index, row in df.iterrows(): #df.iloc[:100].iterrows():
    link = "https://www.immoscout24.ch/en/d/" + str(row['propertyDetails_id'])
    href = '<a href="' + link + '" target="_blank">' + row["propertyDetails_title"].replace("'", r"\'") + ' - ' + row["propertyDetails_priceFormatted"] + '</a>'
    folium.Marker([row["propertyDetails_latitude"], row["propertyDetails_longitude"]], popup=href).add_to(props_map)
props_map.save("props_map_"+file+".html")
props_map

### Heatmap of all properties of this filtered dataset

In [12]:
props_heatmap = []
props_heatmap = folium.Map(location=[47.208835, 7.532291],
                           zoom_start=10)

#todo, show price as heatmap!
props_heatmap.add_child(plugins.HeatMap([[float(row["propertyDetails_latitude"]), float(row["propertyDetails_longitude"])] for index, row in df.iterrows()]))
props_heatmap.save("heatmap_"+file+".html")
props_heatmap


# Hedonische Qualitätsberechnung

- Zu den Eigenschaften der Objekte zählen 
  - Strukturvariabeln (z.B. Anzahl Zimmer, Nettowohnfläche, Alter), 
  - Nutzungsvariabeln (Erst-, Zweitwohnung) und 
  - Lagevariablen (z.B. Distanz zum ÖV, Sicht, Exposition, Lärmbelastung). 


In [13]:
#set display row option to show all attributes of df
pd.set_option('display.max_rows', 1000)



display(df.loc[4934218])

df.loc[4934218]['propertyType']
df.loc[4934218]['numberOfRooms']


#Objekttyp
--> propertyType
#Strukturvariablen 
--> numberOfRooms, living (Wohnfläche), surfaceProperty (Grundstücksfläche), surfaceUsable(Nutzfläche) 
  , volume (Kubatur)
  #Altersentwertung 
  , yearBuilt, yearRenovated, newBuilding (also yearBuilt=this year..)
#Lagevariablen
--> routeDurationSecCity (time until next bigger city (choosen from user)), routeDistanceMetersCity
  , routeDurationSecWork, routeDistanceMetersWork
    #Sicht, Exposition, Lärmbelastung


SyntaxError: invalid syntax (<ipython-input-13-3c0549146805>, line 13)